# 1- Loads `analytics_table.parquet`

In [39]:
import sys
import logging
from pathlib import Path
import pandas as pd

ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT / 'src'))

from data_workflow.viz import bar_sorted, time_line, histogram_chart, save_fig
from data_workflow.utils import bootstrap_diff_means
from data_workflow.config import make_paths

logger = logging.getLogger(__name__)
paths = make_paths(ROOT)

# 2- Performs data audit (rows, dtypes, missingness)

In [40]:
df = pd.read_parquet(paths.processed / "analytics_table.parquet")


print(f"Rows: {len(df)}")
print(f"Columns: {len(df.columns)}")
print(f"Data Types:\n {df.dtypes}")
print(f"Missing Values:\n {df.isna().sum().sort_values(ascending=False).head(10)}")
df.head()


Rows: 5
Columns: 17
Data Types:
 order_id                   string[python]
user_id                    string[python]
amount                            Float64
quantity                            Int64
created_at            datetime64[ns, UTC]
status                             object
amount__isna                         bool
quantity__isna                       bool
created_at_day                    float64
created_at_year                   float64
created_at_month                  float64
created_at_dow                    float64
created_at_hour                   float64
country                            object
signup_date                        object
amount_winsorizes                 Float64
amount__is_outlier                boolean
dtype: object
Missing Values:
 created_at_day        1
created_at_year       1
amount_winsorizes     1
created_at_hour       1
created_at_dow        1
created_at_month      1
amount__is_outlier    1
created_at            1
quantity              1
amount

,order_id,user_id,amount,quantity,created_at,status,amount__isna,quantity__isna,created_at_day,created_at_year,created_at_month,created_at_dow,created_at_hour,country,signup_date,amount_winsorizes,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,paid,False,False,1.0,2025.0,12.0,0.0,10.0,SA,2025-11-15,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,False,False,1.0,2025.0,12.0,0.0,11.0,SA,2025-11-20,8.135,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,refund,True,False,2.0,2025.0,12.0,1.0,9.0,AE,2025-11-22,<NA>,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,paid,False,True,3.0,2025.0,12.0,2.0,14.0,SA,2025-11-15,25.0,False
4,A0005,0004,100.0,1,NaT,paid,False,False,NaN,NaN,NaN,NaN,NaN,SA,2025-11-25,97.75,True


# 3- Answer Questions:

# What is total revenue by country?

In [41]:
revenue_by_country = (
    df.groupby("country")
        .agg(total_revenue=("amount", "sum"))
        .reset_index()
        .sort_values(by="total_revenue", ascending=False)
)


fig = bar_sorted(
    revenue_by_country,
    x="country",
    y="total_revenue",
    title="Total Revenue by Country",
)
save_fig(fig, ROOT/ "reports"/ "figures" / "revenue_by_country.png")

fig

# Interpretations:
Most of the Revenue comes from customers in Saudi Arabia(SA), while in United Arab Emirates(AE) it shows Zero total_revenue in this dataset.

# Caveats:
This dataset is small and may not shows real business performance. Also, (AE) total_revenue shows Zero maybe due to missing values.

# How does Total Revenue changes over time?

In [42]:
revenue_time = (
    df.groupby("created_at_day")["amount"]
    .sum()
    .reset_index()
    .rename(columns={"amount": "daily_revenue"})
    .sort_values(by="created_at_day")
)
        
revenue_time["created_at_day"] = revenue_time["created_at_day"].astype("Int64")
fig = time_line(
    revenue_time,
    x="created_at_day",
    y="daily_revenue",
    title="Daily Revenue Over Time",
)
save_fig(fig, ROOT/ "reports"/ "figures" / "daily_revenue_over_time.png")

fig

# Interpretations:
Daily revenue shows variations across days:
- It was 20.5 in Day1, then it fails down to 0 in Day2 due to a refund, after that it raised up to 25 on Day3.
- Order no. A0005 is not included because the Day is NAN.

# Caveats:
- The datast is too small (only 3 days), so trends are not reliable.
- Refunded transaction may distort the pattern

# Distribution of Order amounts:

In [43]:
amount_summary = df["amount"].describe()
amount_summary


fig = histogram_chart(
    df,
    x="amount",
    nbins=20,
    title="Distribution of Order Amounts",
)
save_fig(fig, ROOT/ "reports"/ "figures" / "amount_distribution.png")
print(amount_summary)
fig

count          4.0
mean        36.375
std      43.022039
min            8.0
25%         11.375
50%          18.75
75%          43.75
max          100.0
Name: amount, dtype: Float64


# Interpretations:
The distribution of order amounts is right-skewed. Most orders are small below 30, while one order is large (100). due to this large value the mean is pulled above the median.

# Caveats:
The dataset contains only 4 non-missing values, so the distribution should not be generalized. 

# 4- bootstrap comparison

In [55]:
SA = df.loc[df["country"] == "SA", "quantity"]
AE = df.loc[df["country"] == "AE", "quantity"]

result = bootstrap_diff_means(SA, AE, n_boot=10000)

result

{'diff_mean': 0.33333333333333326,
 'ci_lower': np.float64(0.0),
 'ci_upper': np.float64(1.0)}